In [2]:
import os

current_dir_path = os.getcwd()
current_file_path = os.path.abspath(current_dir_path)
current_dir_path = os.path.dirname(current_file_path)
parent_dir_path = os.path.dirname(current_dir_path)

os.chdir(current_dir_path)

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN

from tdamapper.core import MapperAlgorithm
from tdamapper.cover import CubicalCover, BallCover, KNNCover, TrivialCover
from tdamapper.plot import MapperLayoutInteractive, MapperLayoutStatic
from tdamapper.clustering import FailSafeClustering
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
import scipy.cluster.hierarchy as shc

from functions import *

from sklearn.cluster import DBSCAN, KMeans
from sklearn import metrics

In [3]:
data1 = pd.read_csv("./Data/NPA_TMA2_1.csv", low_memory=False)[:-2]
data2 = pd.read_csv("./Data/NPA_TMA2_2.csv", low_memory=False)[:-2]
data3 = pd.read_csv("./Data/NPA_TMA2_3.csv", low_memory=False)[:-2]
data4 = pd.read_csv("./Data/NPA_TMA2_4_new.csv", low_memory=False)[:-2]
data5 = pd.read_csv("./Data/NPA_TMA2_5.csv", low_memory=False)[:-2]
data6 = pd.read_csv("./Data/NPA_TMA2_6.csv", low_memory=False)[:-2]

dataA2 = pd.concat([data1, data2, data3, data4, data5, data6], ignore_index=True)

dataA1 = pd.read_csv("./Data/NPA_TMA1_new.csv")[:-2]

In [51]:
dataA2[dataA2['當事者順位'] == 1].shape

(167918, 51)

In [52]:
dataA1[dataA1['當事者順位'] == 1].shape

(787, 51)

In [14]:
select_lst = [
    '光線名稱', '道路類別-第1當事者-名稱', '速限-第1當事者',
    '道路型態大類別名稱', '事故位置大類別名稱',  '號誌-號誌種類名稱',
    '車道劃分設施-分向設施大類別名稱', '車道劃分設施-分道設施-快車道或一般車道間名稱',
    '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',
    '事故類型及型態大類別名稱',
    '死亡受傷人數', '經度', '緯度',
]

In [70]:
def preprocess(input_data, select_lst, sample = 592):
    sample_data = input_data[input_data['當事者順位'] == 1].reset_index(drop=True, inplace=False).sample(sample).reset_index(drop=True)
    dataA = sample_data[select_lst]
    
    death_injury_data = split_death_injury(dataA['死亡受傷人數'])
    dist_df = pd.concat([dataA, death_injury_data], axis=1)
    dist_df.drop(columns=['死亡受傷人數'], inplace=True)
    
    return dist_df, sample_data

def get_full_info(mapper_plot):
    x = vars(mapper_plot._MapperLayoutInteractive__fig)['_data_objs'][1]['x']
    y = vars(mapper_plot._MapperLayoutInteractive__fig)['_data_objs'][1]['y']
    z = vars(mapper_plot._MapperLayoutInteractive__fig)['_data_objs'][1]['z']

    threeDimData = pd.DataFrame({'x': x, 'y': y, 'z': z})

    import re
    data_tuple = vars(mapper_plot._MapperLayoutInteractive__fig)['_data_objs'][1]['text']

    data = []
    for item in data_tuple:
        color = int(re.search(r'color: (\d+)', item).group(1))
        node = int(re.search(r'node: (\d+)', item).group(1))
        size = int(re.search(r'size: (\d+)', item).group(1))
        data.append({'color': color, 'node': node, 'size': size})
    component_info = pd.DataFrame(data)

    full_info = pd.concat([component_info, threeDimData], axis=1)

    mp_content_origin = vars(mapper_plot._MapperLayoutInteractive__graph)['_node']

    mp_content = pd.DataFrame.from_dict(mp_content_origin, orient='index')
    mp_content.reset_index(inplace=True)
    mp_content.rename(columns={'index': 'node'}, inplace=True)

    full_info = pd.merge(full_info, mp_content, on=['node', 'size'], how='inner')
    full_info.iloc[:, 3:6]
    
    return full_info

def resample_plot(A1, A2, selected, p_age = False):
    dist_dfA1 = preprocess(dataA1, selected, sample = 59)
    dist_dfA2 = preprocess(dataA2, selected, sample = 5920)
    # dist_dfA1 = preprocess(dataA1, selected, sample = 78)
    # dist_dfA2 = preprocess(dataA2, selected, sample = 16000)

    rbind_data = pd.concat([dist_dfA1[0], dist_dfA2[0]], axis=0, ignore_index=True)
    
    rbind_data.loc[rbind_data['受傷'] > 1, '受傷'] = 2
    rbind_data['速限-第1當事者'] = rbind_data['速限-第1當事者'].apply(lambda x: 1 if x > 60 else 0)
    if p_age == True:
        rbind_data = process_age(rbind_data)

    dist_df = process_data(rbind_data)
    scaler = StandardScaler()

    full_dist = pd.DataFrame(scaler.fit_transform(dist_df), columns = dist_df.columns)
    X1 = full_dist.drop(['受傷', '死亡', '經度', '緯度'], axis=1).to_numpy()

    lens1 = PCA(10).fit_transform(X1)

    mapper_algo1 = MapperAlgorithm(
        cover = CubicalCover(
            n_intervals = 3,
            overlap_frac = 0.3
        ),
        clustering = FailSafeClustering(
            clustering = AgglomerativeClustering(3, metric='euclidean', linkage='ward'),
            verbose = False)
    )
    mapper_graph1 = mapper_algo1.fit_transform(X1, lens1)

    mapper_plot1 = MapperLayoutInteractive(
        mapper_graph1,
        colors = rbind_data[['速限-第1當事者']].to_numpy(),
        cmap = 'jet',
        agg = np.nanmean,
        # agg = most_frequent_nonan,
        dim = 3,
        iterations = 30,
        seed = 6,
        width = 800,
        height = 500)
    
    return mapper_plot1

In [72]:
mapper_plot1 = resample_plot(dataA1, dataA2, select_lst)
full_info = get_full_info(mapper_plot1)
calinski_data = get_calinski_from_db(full_info, 0.05)
print(calinski_data[0])

labels = calinski_data[3]
db = calinski_data[2]
n_clusters_ = calinski_data[4]

print(n_clusters_)

# fig_mean1 = mapper_plot1.plot()
# fig_mean1.show(config={'scrollZoom': True})

21.63711103516419
2
